# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [87]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [88]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [89]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

neighborhood_coords_csv = Path("Data/neighborhoods_coordinates.csv")
neighborhood_coords = pd.read_csv(neighborhood_coords_csv)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [90]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    mean_housing_units = sfo_data['housing_units'].groupby('year').mean()
    mean_housing_units_bar = px.bar(mean_housing_units,
    x=mean_housing_units.index,
    y="housing_units",
    title="Housing Units in San Francisco from 2010 to 2016",
    labels={"housing_units":"Housing units"},
    range_y=[370000,387000],
)
    return mean_housing_units_bar


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    avg_mon_rent = sfo_data.groupby(["year"])["gross_rent"].mean()
    avg_mon_rent_line= px.line(avg_mon_rent,
    x=avg_mon_rent.index,
    y='gross_rent' ,
    title="Average Monthly Rent by Year", 
    labels={'gross_rent':'Monthly Rent'}
)
    return avg_mon_rent_line



def average_sales_price():
    """Average Sales Price Per Year."""
    avg_sale_sqft = sfo_data.groupby(["year"])["sale_price_sqr_foot"].mean()
    avg_sale_sqft_line = px.line(avg_sale_sqft,
    x=avg_sale_sqft.index,
    y='sale_price_sqr_foot',
    title="Average Sales Price per Square Foot by Year", 
    labels={'sale_price_sqr_foot':'Housing Units Price per Square Foot'}
)
    return avg_sale_sqft_line



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    avg_prices_yr_neighborhood = sfo_data.groupby(["year", "neighborhood"]).mean()
    avg_price_sqft_hvplot = avg_prices_yr_neighborhood.hvplot(
    x="year",
    y="sale_price_sqr_foot",
    kind="line",
    ylabel="Average Sale Price per sqft",
    groupby="neighborhood"
)
    return avg_price_sqft_hvplot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_10_exp = sfo_data.groupby(["neighborhood"]).mean().nlargest(10, "sale_price_sqr_foot").sort_values(ascending=False, by="sale_price_sqr_foot")
    top_10_exp_bar = px.bar(top_10_exp,
    x= top_10_exp.index,
    y='sale_price_sqr_foot',
    title="Top 10 Expensive Neighborhoods in San Francisco",
    labels={'sale_price_sqr_foot':'Average Sale Price Per Square Foot'}
)
    return top_10_exp_bar


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    comparison_hvplot = sfo_data.groupby(["year", "neighborhood"]).mean()
    comparison_hvplot = comparison_hvplot.hvplot.bar(
    x="year",
    y=["gross_rent", "sale_price_sqr_foot"],
    value_label="Num Housing Units",
    groupby="neighborhood",
    title="SFO Average Price per Sqft vs Average Monthly Rent",
    rot=45,
    height=400,
)
    return comparison_hvplot
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    # YOUR CODE HERE!



def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!



def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!


def sunburst():
    """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [91]:
# Create a Title for the Dashboard
tab1 = pn.pane.Markdown(
    """
    #Welcome to San Francisco (SFO)!
    ##Over the past decade San Francisco has been growing steadily, so let's take a look at San Francisco's real estate information!
    ###The next few tabs will include plots that will help you decide which SFO locations are the most lucrative to invest in.
  
    """
)
# Create a tab layout for the dashboard
tab_2 = pn.Row(
    "#San Francisco Rental Analysis - 2010 to 2016",
   housing_units_per_year(),
   average_gross_rent(),
   average_sales_price()
)
tab_3 = pn.Row(
    "# San Francisco Neighborhood Analysis - 2010 to 2016",
    top_most_expensive_neighborhoods(),
)
tab_4 = pn.Row(
    "# San Francisco Average Price per Sqft vs Average Monthly Rent Comparison",
    most_expensive_neighborhoods_rent_sales(),
)
# Create the dashboard
sfo_db = pn.Tabs(
    ("San Francisco: Real Estate Investments",tab_1),
    ("Rental Analysis",tab_2),
    ("Most Expensive Neighborhoods in San Francisco", tab_3),
    ("Neighborhood Average Prices per Sqft vs. Average Monthly Rent",tab_4),
)

## Serve the Panel Dashboard

In [92]:
# Serve the# dashboard
sfo_db.servable()

Tabs
    [0] Row
        [0] Markdown(str)
        [1] Plotly(Figure, relayout_data={'autosize': True}, viewport={'xaxis.range': [2009.5, ...})
        [2] Plotly(Figure, relayout_data={'autosize': True}, viewport={'xaxis.range': [2010, ...})
        [3] Plotly(Figure, relayout_data={'autosize': True}, viewport={'xaxis.range': [2010, ...})
    [1] Row
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)
        [2] Plotly(Figure, visible=False)
        [3] Plotly(Figure, visible=False)
    [2] Row
        [0] Markdown(str)
        [1] Plotly(Figure, visible=False)
    [3] Row
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [86]:
# housing_units_per_year()

In [53]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()